This notebook heavily refers to the following resources:
1. https://www.youtube.com/watch?v=viZrOnJclY0&ab_channel=StatQuestwithJoshStarmer

Word embedding is the process of converting a word into a vector that can be processed by a neural network. Unlike a tokenizer, where the input is simply being converted to a random token, the embedding process aims to create an embedded dictionary where words of similar meaning will be ``closer" when embedded into a vector. Words may also carry different meaning, therefore we hope to assign more than 1 number to describe different words. In this notebook, we are going to train our own word embedder using neural network. 

Consider the following training data
1. Fruits are tasty.
2. Cakes are tasty.

In our training data, there are a total of 4 unique words: fruits, are, tasty, cakes. Using the one hot encoding method, we can assign each unique words with a one vector, and the output of the word with another vector. 

Input:
1. fruits - [1, 0, 0, 0]
2. are    - [0, 1, 0, 0]
3. tasty  - [0, 0, 1, 0]
4. cakes  - [0, 0, 0, 1]

Label:
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]
[0, 1, 0, 0]

Consider the following 3 training data:
1. Fruits are tasty.
2. Fruits are delicious.
3. Fruits are tasty and delicious

The word 'are' has 2 possible outputs, tasty and delicious. One hot encoding is unable to encode this relation properly. This becomes a multi-label encoding scheme, where each sample can be associated with multiple binary labels, with each label indicating the presence or absence of a category. 

Label :
[0, 1, 0, 0, 0]
[0, 0, 1, 1, 0]
[0, 0, 0, 0, 1]
[0, 0, 0, 0, 1]
[0, 0, 1, 1, 0]




In pytorch, this is done through the torch.tensor() function


In [5]:
import torch

inputs = torch.tensor([[1., 0., 0., 0.],
                       [0., 1., 0., 0.],
                       [0., 0., 1., 0.],
                       [0., 0., 0., 1.]])

labels = torch.tensor([[0., 1., 0., 0.],
                       [0., 0., 1., 0.],
                       [0., 0., 0., 1.],
                       [0., 1., 0., 0.]])

dataset = torch.utils.data.TensorDataset(inputs, labels)
dataloader = torch.utils.data.DataLoader(dataset)

In [3]:
class wordembedding():

SyntaxError: incomplete input (2859970609.py, line 1)